<a href="https://colab.research.google.com/github/gd-Sahat/TL-LEO-POD/blob/main/starlink_eph_parser.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import pandas as pd
import zipfile
import re
from datetime import datetime, timedelta
import os
import glob

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:


def parse_modified_itc(content, source_name=None):
    """
    Parse Modified ITC ephemeris file format.
    Args:
        content (str): File content as a string
        source_name (str): Optional filename to tag each record
    Returns:
        pd.DataFrame: DataFrame with ephemeris data
    """
    lines = content.strip().split('\n')

    # Skip header (3 lines) + UVW marker
    data_lines = lines[4:]
    data = []
    block_size = 4
    num_blocks = len(data_lines) // block_size

    for i in range(num_blocks):
        block = data_lines[i*block_size:(i+1)*block_size]
        ts_line = block[0].split()
        # Parse timestamp
        year, doy = int(ts_line[0][:4]), int(ts_line[0][4:7])
        hour, minute = int(ts_line[0][7:9]), int(ts_line[0][9:11])
        second = int(ts_line[0][11:13])
        micros = int(ts_line[0][14:17]) * 1000
        base = datetime(year, 1, 1) + timedelta(days=doy-1)
        timestamp = base.replace(hour=hour, minute=minute, second=second, microsecond=micros)

        x, y, z = map(float, ts_line[1:4])
        vx, vy, vz = map(float, ts_line[4:7])

        cov_vals = []
        for line_cov in block[1:4]:
            cov_vals.extend(map(float, line_cov.split()))

        record = {
            "timestamp": timestamp, "x": x, "y": y, "z": z,
            "vx": vx, "vy": vy, "vz": vz,
            "source_file": source_name
        }
        for idx, val in enumerate(cov_vals, start=1):
            record[f"cov_{idx}"] = val

        data.append(record)

    return pd.DataFrame(data)

def parse_zip_modified_itc(zip_path):
    """
    Parse all .txt ephemeris files inside one ZIP,
    tagging rows with sat_id (from filename) and zip_file.
    """
    dfs = []
    id_pattern = re.compile(r'(STARLINK-\d+|\d{7,})')
    with zipfile.ZipFile(zip_path, 'r') as z:
        for fname in z.namelist():
            if not fname.lower().endswith('.txt'):
                continue
            # Extract satellite ID from filename
            m = id_pattern.search(fname)
            sat_id = m.group(0) if m else os.path.splitext(os.path.basename(fname))[0]

            # Read & parse the file
            with z.open(fname) as f:
                content = f.read().decode('utf-8')
            df = parse_modified_itc(content, source_name=fname)

            # Tag it
            df['sat_id']  = sat_id
            df['zip_file'] = os.path.basename(zip_path)
            dfs.append(df)

    return pd.concat(dfs, ignore_index=True)

def parse_multiple_zips(zip_dir):
    """
    Walk through zip_dir for all .zip files and parse them.
    """
    all_dfs = []
    for zp in glob.glob(os.path.join(zip_dir, '*.zip')):
        all_dfs.append(parse_zip_modified_itc(zp))
    return pd.concat(all_dfs, ignore_index=True)

zip_dir = '/content/drive/MyDrive/STARLINK-EPH'
combined_df = parse_multiple_zips(zip_dir)
combined_df.to_csv('all_starlink_ephemerides.csv', index=False)

In [ ]:
starlink_tles_df = pd.read_csv('starlink_tles.csv')

In [ ]:
starlink_tles_df

,name,line1,line2,epoch_datetime
0,0 STARLINK-1008,1 44714U 19074B 25170.57310755 .00000440 0...,2 44714 53.0561 287.7961 0001413 93.6781 266...,2025-06-19 13:45:16.492320
1,0 STARLINK-1010,1 44716U 19074D 25170.02404143 .00086974 0...,2 44716 53.0558 278.0056 0005786 290.8872 69...,2025-06-19 00:34:37.179552
2,0 STARLINK-1010,1 44716U 19074D 25170.02404143 .00086974 0...,2 44716 53.0558 278.0056 0005786 290.8872 69...,2025-06-19 00:34:37.179552
3,0 STARLINK-1010,1 44716U 19074D 25170.54009606 .00068862 0...,2 44716 53.0557 275.5311 0005884 293.1051 66...,2025-06-19 12:57:44.299584
4,0 STARLINK-1011,1 44717U 19074E 25170.37959632 -.00000032 0...,2 44717 53.0555 308.6645 0001099 86.4854 273...,2025-06-19 09:06:37.122048
...,...,...,...,...
17621,0 STARLINK-34218,1 64338U 25124X 25170.91668981 -.00949357 0...,2 64338 43.0034 174.6893 0001277 268.0436 308...,2025-06-19 22:00:01.999584
17622,0 STARLINK-34017,1 64339U 25124Y 25170.50002315 -.00943914 0...,2 64339 43.0030 177.2362 0002415 261.7194 102...,2025-06-19 12:00:02.000160
17623,0 STARLINK-34017,1 64339U 25124Y 25170.58335648 -.00956287 0...,2 64339 43.0032 176.7235 0002386 262.1294 217...,2025-06-19 14:00:01.999872
17624,0 STARLINK-34017,1 64339U 25124Y 25170.83335648 -.00975077 0...,2 64339 43.0034 175.1881 0002483 258.8487 206...,2025-06-19 20:00:01.999872
